# Environment preparation

In [1]:
# install necessary libraries
# pip install nltk keras tensorflow matplotlib pandas scikit-learn

# Sentiment Analysis over Movie Reviews

## Text normalization

## Feature extraction

## Model training

## Model report

# Author Identification from News Articles

## Text normalization

## Feature extraction

## Model report

## Model training